In [ ]:
# settting up in colab and importing packages. 

from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/My Drive/Dissertation'
from tqdm import tqdm
import pandas as pd
import ast

# reading the files
df=pd.read_csv("reviews_with_visible_comments.csv")
reviews_list=pd.read_csv("FPS_reviews.csv")

# converting it to lists
df["order_of_visible_comments"]=[(ast.literal_eval(df["order_of_visible_comments"][i])) for i in (range(len(df)))]
df["order_of_times"]=[(ast.literal_eval(df["order_of_times"][i])) for i in (range(len(df)))]

# ensuring the elements of the list are converted to its appropriate integers or strings respectively
def convert_to_int(old_list):
  new_list=[]
  for i in range(len(old_list)):
    temp_list=[]
    for j in range(len(old_list[i])):
      temp_list.append(int((old_list[i])[j]))
    new_list.append(temp_list)
  return(new_list)

def convert_to_str(old_list):
  new_list=[]
  for i in range(len(old_list)):
    temp_list=[]
    for j in range(len(old_list[i])):
      temp_list.append(str((old_list[i])[j]))
    new_list.append(temp_list)
  return(new_list)

df["order_of_times"]=convert_to_int(list(df["order_of_times"]))
df["order_of_visible_comments"]=convert_to_str(list(df["order_of_visible_comments"]))
reviews_list["game_id"]=[str(i) for i in (list(reviews_list["game_id"]))]
reviews_list["timestamp_created"]=[int(i) for i in (list(reviews_list["timestamp_created"]))]
reviews_list["recommendation_id"]=[str(i) for i in (list(reviews_list["recommendation_id"]))]

# get unobserved reviews for the past 30 days 
def get_temp_df(df,reviews_list):
  invisible_comments_list=[]
  for i in tqdm(range(len(df))):
    #bin=int((df["visible_comment_date_bin"])[i])
    end_int=int(df.timestamp_created[i])
    #start=int(end_int-(bin*86400))
    start=int(end_int-(30*86400))
    game_id=str(df.game_id[i])
    temp_dates=reviews_list[(reviews_list['game_id']==game_id)] # get reviews only from the same game
    temp_dates = temp_dates[temp_dates['timestamp_created'].between(start,(end_int-1))] # limit to only re-simulations that display reviews of the past 30 days for consistency. 
    temp_dates=temp_dates.sort_values("timestamp_created",ascending=False) # sort by most recent to earliest
    if len(temp_dates)<10: # only interested if the reviewer has 10 or more unobserved comments
      invisible_comments_list.append([])
    else:
      observable_list=df.order_of_visible_comments[i] 
      temp_dates_list=list(temp_dates.recommendation_id)
      unobservable_temp_dates_list = [x for x in temp_dates_list if x not in observable_list] # filtering games observed vs unobserved
      invisible_comments_list.append(unobservable_temp_dates_list) 
  return(invisible_comments_list)


df["unobservable_comments_list"]=get_temp_df(df,reviews_list)
# df.to_csv('FPS_reviews_complete.csv')

# removing games with <10 recently-sorted reviews
index_of_less_than_10=[]
for i in tqdm(range(len(df))):
  #print(df["order_of_visible_comments"])[i])
  if len(df["unobservable_comments_list"][i])<10:
    index_of_less_than_10.append(i)
  else:
    continue
df=df.drop(index_of_less_than_10)
#df.to_csv('FPS_reviews_complete.csv')
df

# rearranging the files so as to save as pickle file.  
import pickle
df["index"]=[i for i in range(len(df))]
df=df.set_index("index")
temp_list_observable=list(df["order_of_visible_comments"])
temp_list_unobservable=list(df["unobservable_comments_list"])
recommendation_id_list=list(df["recommendation_id"])


# saving as a pickle so as to save space (ie takes less space than a csv file)
with open("unobservable_comments_list", "wb") as fp1:
  pickle.dump(temp_list_unobservable, fp1)

with open("observable_comments_list", "wb") as fp2:
  pickle.dump(temp_list_observable, fp2)

with open("recommendation_id", "wb") as fp3:
  pickle.dump(recommendation_id_list, fp3)